In [1]:
import ragas, os
import pandas as pd
import os, nest_asyncio
from datasets import Dataset
os.chdir('/Users/1zuu/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/LLM RESEARCH 2024/LLM in Production/01-diy-llm-qa-bot')
nest_asyncio.apply() 

In [2]:
from src.qabot_pipe import *

Using Groq LLM ....
Using HuggingFace Embedding ....


/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
qabot = QABot(llm=llm)
qabot.get_answer("What are the Eligibility criteria to open a Fixed Deposit?")

Total nodes: 16
Loading Finance Index ...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'The eligibility criteria to open a Fixed Deposit include being a citizen of Sri Lanka above 18 years of age holding a valid National Identity Card (NIC), a valid Passport, or a Driving License. Additionally, Sri Lankan citizens residing temporarily outside Sri Lanka, individuals holding dual citizenship, and non-residents who possess a resident visa are also eligible.'

In [4]:
def make_response(question):
    response = qabot.get_answer(question)
    if response is None:
        return ""
    return response

def load_eval_data():
    df = pd.read_csv(path_config.eval_dataset_path)
    df['response'] = df['question'].apply(make_response)

    df = df.rename(columns={
                            'context': 'contexts', 
                            'answer': 'ground_truth',
                            'response': 'answer'
                            })
    df['contexts'] = df['contexts'].apply(lambda x: [x])
    eval_data = Dataset.from_dict(df)
    return eval_data

def ragas_evaluate():
    eval_data = load_eval_data()
    result_json = ragas.evaluate(
                                dataset=eval_data,
                                metrics=[
                                        ragas.metrics.faithfulness,
                                        ragas.metrics.answer_relevancy,
                                        ragas.metrics.context_precision,
                                        ragas.metrics.context_recall
                                        ]
                            )
    result_df = result_json.to_pandas()
    result_df = result_df.fillna(0)
    return result_df

def evaluation_pipeline():
    result_df = ragas_evaluate()
    result_df.to_csv(path_config.eval_results_path, index=False)

def calculate_avg_metrics():
    if not os.path.exists(path_config.eval_results_path):
        evaluation_pipeline()

    result_df = pd.read_csv(path_config.eval_results_path)
    result_df = result_df[[
                        'faithfulness',
                        'answer_relevancy',
                        'context_precision',
                        'context_recall'
                        ]]
    avg_metrics = result_df.mean()
    return avg_metrics

In [5]:
calculate_avg_metrics()

faithfulness         0.600000
answer_relevancy     0.852696
context_precision    1.000000
context_recall       1.000000
dtype: float64